In [1]:
!pip3 install -qU pybiolib
import biolib
import pandas as pd
deeptmhmm = biolib.load('DTU/DeepTMHMM')
biolib.login()


2023-06-06 17:52:59,023 | INFO : Loaded project DTU/DeepTMHMM:1.0.24
2023-06-06 17:52:59,025 | INFO : Already signed in


2023-06-05 18:16:43,664 | INFO : Job "82df41eb-a3b0-4083-b253-c52750ab47c2" is starting...
2023-06-05 18:16:51,451 | INFO : Cloud: Initializing
2023-06-05 18:16:51,452 | INFO : Cloud: Pulling images...
2023-06-05 18:16:51,452 | INFO : Cloud: Computing...
Traceback (most recent call last):
  File "predict.py", line 41, in <module>
    fasta_size = os.stat(args.fasta).st_size
FileNotFoundError: [Errno 2] No such file or directory: 'query.fasta'
2023-06-05 18:16:54,014 | INFO : Cloud: Computation finished
2023-06-05 18:16:54,015 | INFO : Cloud: Result Ready


In [16]:
import re

deeptmhmm = biolib.load('DTU/DeepTMHMM')
biolib.utils.STREAM_STDOUT = True
accession_only = True

def make_file(file_name, pep_seq):
    with open(file_name, "w") as file:
        sample_category = pep_seq['Biological sample category']
        protein_name = pep_seq['Protein name']
        normalized_percent = pep_seq['Normalized protein percentage']
        peptide_sequence = pep_seq['Peptide sequence']
        accession_number = pep_seq['Protein accession number']
        
        pattern = r"OX=\d+"
        match = re.search(pattern, protein_name)

        pattern2 = r"GN=\w+"
        match2 = re.search(pattern2, protein_name)

        if not accession_only and match and match2:
            ox_value = match.group()
            gn_value = match2.group()
            oxgn_value = str(ox_value) + " " + str(gn_value)
        else:
            oxgn_value = str(accession_number)            
        
        header = ">Type: " + str(sample_category) + " ||| " + oxgn_value + " ||| " + str(normalized_percent) + "\n"
        file.write(header)
        file.write(peptide_sequence)


# Only a certain number of records can be analyzed at a given time
data = pd.read_excel('output.xlsx', header=0)
data = data.iloc[0:100]


fasta_list = []

# Generate fasta format files for deepTMHMM processing
for index, peptide in data.iterrows():
    
    make_file("fasta_files/query"+str(index)+".fasta", peptide)
    fasta_list.append("fasta_files/query"+str(index)+".fasta")

print("List writing complete")

# Send the fasta files to the TMHMM database for processing and save the results
index = 0
with biolib.Experiment('TMHMM_analysis'):
    for fasta_file in fasta_list:
        result = deeptmhmm.cli(args='--fasta '+fasta_file, machine='local')
        result.save_files('transmembrane_results_local5/peptide_row'+str(index))
        index+=1

print("done!")

2023-06-06 20:14:02,765 | INFO : Loaded project DTU/DeepTMHMM:1.0.24
List writing complete
2023-06-06 20:14:08,769 | INFO : Job "bad133a0-316a-4cd4-9939-c62cdfe004ff" is starting...
2023-06-06 20:14:09,062 | INFO : Started compute node
2023-06-06 20:14:10,602 | INFO : Compute Node: Initializing
2023-06-06 20:14:11,065 | INFO : Job "bad133a0-316a-4cd4-9939-c62cdfe004ff" running...
2023-06-06 20:14:12,106 | INFO : Compute Node: Pulling images...
2023-06-06 20:14:12,106 | INFO : Compute Node: Computing...
Running DeepTMHMM on 1 sequence...
Step 1/4 | Loading transformer model...

Step 2/4 | Generating embeddings for sequences...
Generating embeddings: 100% 1/1 [00:00<00:00,  3.38seq/s]

Step 3/4 | Predicting topologies for sequences in batches of 1...
Topology prediction: 100% 1/1 [00:00<00:00, 11.03seq/s]

Step 4/4 | Generating output...
2023-06-06 20:14:21,132 | INFO : Compute Node: Computation finished
2023-06-06 20:14:21,133 | INFO : Compute Node: Result Ready
2023-06-06 20:14:21,797 

In [5]:
'''

samtools = biolib.load('samtools/samtools')


data = pd.read_excel('Malaria_Research_Data.xlsx', header=0)

data = data['Peptide sequence'].head(1)

print(data)

results = []

for peptide in data:
    result = samtools.cli(peptide, blocking=True)
    print(result.get_stdout().decode("utf-8"))
    results.append(result)

# print(results[0].get_stdout().decode("utf-8"))
print("done!")
'''

2023-06-05 19:21:57,355 | INFO : Loaded project samtools/samtools:0.0.79
0    EYILESGAVVLSDK
Name: Peptide sequence, dtype: object
2023-06-05 19:22:01,523 | INFO : Job "f9e9327b-1345-46e5-aff8-68e9e9c26d63" is starting...
2023-06-05 19:22:10,928 | INFO : Cloud: Initializing
2023-06-05 19:22:10,929 | INFO : Cloud: Downloading Source Files...
2023-06-05 19:22:10,929 | INFO : Cloud: Pulling images...
2023-06-05 19:22:10,929 | INFO : Cloud: Computing...
[main] unrecognized command 'EYILESGAVVLSDK'
2023-06-05 19:22:16,059 | INFO : Cloud: Computation finished
2023-06-05 19:22:16,060 | INFO : Cloud: Result Ready
[main] unrecognized command 'EYILESGAVVLSDK'

done!


In [12]:
'''
samtools = biolib.load('samtools/samtools')

def make_file(file_name, pep_seq):
    with open(file_name, "w") as file:
        file.write(pep_seq)

def write_results(file_name, output):
    with open(file_name, "wb") as file:
        file.write(output)


data = pd.read_excel('output.xlsx', header=0)

data = data['Peptide sequence'].head(2)

parse_limit = 50

results = []
fasta_list = []

index = 0
for peptide in data:    
    make_file("fasta_files/query"+str(index)+".fasta", peptide)
    fasta_list.append("fasta_files/query"+str(index)+".fasta")
    index+=1

print("File writing completed")

parsed_list = []
index = 0
while index < len(data):
    temp_list = []
    j=0
    while j < 50 and index < len(fasta_list):
        temp_list.append(fasta_list[index])
        index+=1
        j+=1
    parsed_list.append(temp_list)

print("Parsing completed")
    

    
index = 0
segment = 0
while index < len(data):
    with biolib.Experiment('TMHMM_analysis'+str(segment)):
        for fasta_file in parsed_list[segment]:
            result = samtools.cli(fasta_file, blocking=False)
            results.append(result)
            #result.save_files('transmembrane_results/peptide_row'+str(index))
            index+=1
    current_experiment = biolib.get_experiment('TMHMM_analysis'+str(segment))
    current_experiment.wait()


    result.save_files('transmembrane_results4/peptide_row'+str(7))
    result_index = segment*parse_limit
    for result in results:
        result.save_files('transmembrane_results4/peptide_row'+str(result_index))
        result_index+=1
    
    segment+=1
    
        
'''

'''
index = 0
for cloud_result in results:
    file_name = "transmembrane_results/"+str(index)+".txt"
    write_results(file_name, results[index].get_stdout())
    cloud_result.save_files('transmembrane_results/peptide_row'+str(index))
    index+=1
'''    
    
'''
print(results[0].get_stdout())
print(results[1].get_stdout())
'''

print("done!")

2023-06-06 19:12:27,566 | INFO : Loaded project samtools/samtools:0.0.79
File writing completed
Parsing completed


KeyboardInterrupt: 

In [15]:
'''

import re

deeptmhmm = biolib.load('DTU/DeepTMHMM')
biolib.utils.STREAM_STDOUT = True
samtools = biolib.load('samtools/samtools')


def make_file(file_name, pep_seq):
    with open(file_name, "w") as file:
        sample_category = pep_seq['Biological sample category']
        protein_name = pep_seq['Protein name']
        normalized_percent = pep_seq['Normalized protein percentage']
        peptide_sequence = pep_seq['Peptide sequence']
        accession_number = pep_seq['Protein accession number']
        
        pattern = r"OX=\d+"
        match = re.search(pattern, protein_name)

        pattern2 = r"GN=\w+"
        match2 = re.search(pattern2, protein_name)

        if match and match2:
            ox_value = match.group()
            gn_value = match2.group()
            oxgn_value = str(ox_value) + " " + str(gn_value)
        else:
            oxgn_value = str(accession_number)            
        
        header = ">Type: " + str(sample_category) + " ||| " + oxgn_value + " ||| " + str(normalized_percent) + "\n"
        file.write(header)
        file.write(peptide_sequence)


data = pd.read_excel('output.xlsx', header=0)
data = data.head(10)

results = []

fasta_list = []


for index, peptide in data.iterrows():
    
    make_file("fasta_files/query"+str(index)+".fasta", peptide)
    fasta_list.append("fasta_files/query"+str(index)+".fasta")


print("List writing complete")
    
index = 0
with biolib.Experiment('TMHMM_analysis'):
    for fasta_file in fasta_list:
        result = samtools.cli(args='--fasta '+fasta_file, blocking=False)
        #results.append(result)
        index+=1
        print("run")
experiment_run = biolib.get_experiment('TMHMM_analysis')

all_jobs = experiment_run.get_jobs()

for job in all_jobs:
    job.save_files('results')


print("done!")

'''

2023-06-06 19:52:28,737 | INFO : Loaded project DTU/DeepTMHMM:1.0.24
2023-06-06 19:52:29,334 | INFO : Loaded project samtools/samtools:0.0.79
List writing complete
run
run
run
run
run
run
run
run
run
run
2023-06-06 19:54:24,953 | INFO : Saving 0 files to results...
2023-06-06 19:54:28,933 | INFO : Saving 0 files to results...
2023-06-06 19:54:33,347 | INFO : Saving 0 files to results...
2023-06-06 19:54:37,999 | INFO : Saving 0 files to results...
2023-06-06 19:54:43,133 | INFO : Saving 0 files to results...
2023-06-06 19:54:50,315 | INFO : Saving 0 files to results...
2023-06-06 19:54:54,311 | INFO : Saving 0 files to results...
2023-06-06 19:54:57,991 | INFO : Saving 0 files to results...
2023-06-06 19:55:01,692 | INFO : Saving 0 files to results...
2023-06-06 19:55:05,435 | INFO : Saving 0 files to results...
2023-06-06 19:55:09,617 | INFO : Saving 0 files to results...
2023-06-06 19:55:15,599 | INFO : Saving 0 files to results...
2023-06-06 19:55:20,578 | INFO : Saving 0 files to r

Exception: get_data got not ok response status 404